In [ ]:
import torch

# as 'model' is T5ForConditionalGeneration object
torch.save(model.state_dict(), 'resume_ranking_model.pth')
# Save the model's state dictionary to a .pth file


In [ ]:
!pip install ngrok
!pip install PyMuPDF  # For PDF processing

In [ ]:
!ngrok config add-authtoken 2nQ3pFBFq91lamPkHw4T36dfCKO_6GK37BkB4QxSqLBqUUPvq

In [ ]:
from pyngrok import ngrok
# Terminate any existing tunnels
ngrok.kill()

# Start a new tunnel for port 8501 (Streamlit's default port)
# Use 'addr' instead of 'port' for specifying the address
public_url = ngrok.connect(addr='8501')
print(f"Streamlit public URL: {public_url}")

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib
from transformers import T5ForConditionalGeneration, T5Tokenizer
import fitz  # PyMuPDF for PDF processing

# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Load the pre-trained Random Forest model
clf = joblib.load('resume_ranking_model.pkl')  # Adjust the path as necessary

def predict_score(resume_features):
    """Predict the hiring score based on resume features."""
    # Get the prediction probability for the 'hired' class
    score = clf.predict_proba([resume_features])[0][1]  # Get probability of being hired
    return score

def explain_ranking(resume_features):
    """Provide insights into ranking based on features."""
    explanation = f"Similarity: {resume_features[0]}, Experience: {resume_features[1]} years, Skills Match: {resume_features[2]}"
    return explanation

def extract_text_from_pdf(pdf_file):
    """Extract text from a PDF file."""
    text = ""
    with fitz.open(pdf_file) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Streamlit App
st.title('AI-Powered Resume Ranking System')

# Upload Job Description
uploaded_jd = st.file_uploader("Upload Job Description (txt file)", type=["txt"])
uploaded_cvs = st.file_uploader("Upload CVs (pdf file)", type=["pdf"], accept_multiple_files=True)

if uploaded_jd and uploaded_cvs:
    # Read the job description
    jd_text = uploaded_jd.read().decode("utf-8")

    results = []

    # Process each PDF CV
    for uploaded_cv in uploaded_cvs:
        cv_text = extract_text_from_pdf(uploaded_cv)

        # we can further process cv_text to extract specific information, e.g., experience, skills
        # For this example, we'll create mock values for demonstration
        similarity = np.random.uniform(0.5, 1.0)  # Replace with actual similarity calculation
        experience_years = 5  # Replace with actual extracted value
        skills_match = 1  # Replace with actual extracted value

        # Create feature set
        resume_features = [similarity, experience_years, skills_match]

        # Predict the score
        score = predict_score(resume_features)

        # Generate feedback using T5
        input_text = f"Generate feedback for the resume with similarity {similarity}, experience {experience_years}, skills match {skills_match}."
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        outputs = model.generate(input_ids)
        feedback = tokenizer.decode(outputs[0], skip_special_tokens=True)

        results.append((uploaded_cv.name, score))

    # Display results
    results_df = pd.DataFrame(results, columns=["CV File Name", "Score"])
    st.write("Ranking Results:")
    st.dataframe(results_df)

    # Allow download of results
    st.write("Download Ranking Results:")
    st.download_button("Download CSV", results_df.to_csv(index=False), "ranked_resumes.csv")


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501